# 作業三
## 1.資料前處理 data preprocessing

### a. 讀取csv前10000筆，保留text與score

In [1]:
import pandas as pd 

In [2]:
df = pd.read_csv(r'Reviews.csv')

In [3]:
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [4]:
sample_data = df[['Text','Score']][:10000]
len(sample_data)

10000

In [5]:
sample_data.head()

,Text,Score
0,I have bought several of the Vitality canned d...,5
1,Product arrived labeled as Jumbo Salted Peanut...,1
2,This is a confection that has been around a fe...,4
3,If you are looking for the secret ingredient i...,2
4,Great taffy at a great price. There was a wid...,5


### 將 "Score" 欄位內值大於等於4的轉成1(positive)，其餘轉成0(negative) 

In [6]:
sample_data['Score'] = sample_data['Score'].map(lambda x: 1 if x>=4 else 0) #此 map是pandas map不是一般python map

In [7]:
sample_data.head()

,Text,Score
0,I have bought several of the Vitality canned d...,1
1,Product arrived labeled as Jumbo Salted Peanut...,0
2,This is a confection that has been around a fe...,1
3,If you are looking for the secret ingredient i...,0
4,Great taffy at a great price. There was a wid...,1


### 將text欄位內的文字利用分割符號切割

In [8]:
#sample_data['Text'] = sample_data['Text'].map(lambda x: x.split())
#sample_data['Text'][0]

### b. 去除停頓詞stop words 

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

In [10]:
sample_data['Text'][0]

'I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most.'

In [11]:
vectorizer = CountVectorizer(stop_words='english')
count_vector = vectorizer.fit_transform(sample_data['Text'])

In [12]:
transformer = TfidfTransformer()
tfidf_vector = transformer.fit_transform(count_vector)

TfidfTransformer(CountVectorizer(input)) == TfidfVectorizer(input)

In [13]:
tfidf = pd.DataFrame(tfidf_vector.toarray(),columns=vectorizer.get_feature_names())
tfidf['product'].head(20) #隨便找一個詞來看一下

0     0.202600
1     0.168716
2     0.000000
3     0.000000
4     0.000000
5     0.000000
6     0.000000
7     0.000000
8     0.000000
9     0.000000
10    0.000000
11    0.068892
12    0.000000
13    0.000000
14    0.000000
15    0.000000
16    0.000000
17    0.000000
18    0.040345
19    0.000000
Name: product, dtype: float64

In [14]:
df_tfidf = pd.concat([sample_data,tfidf],join='inner',axis=1)

In [15]:
df_tfidf.head()

,Text,Score,00,000,0003,000kwh,002,008,0100,0174,...,zon,zoo,zoom,zotz,zucchini,zuke,zukes,zupas,zuppa,ît
0,I have bought several of the Vitality canned d...,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Product arrived labeled as Jumbo Salted Peanut...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,This is a confection that has been around a fe...,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,If you are looking for the secret ingredient i...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Great taffy at a great price. There was a wid...,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### w2v實作

In [16]:
from gensim.parsing.preprocessing import remove_stopwords

In [17]:
print(sample_data['Text'][0])
print(remove_stopwords(sample_data['Text'][0]))

I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most.
I bought Vitality canned dog food products good quality. The product looks like stew processed meat smells better. My Labrador finicky appreciates product better most.


In [18]:
w2v_words = []
for Texts in sample_data['Text']:
    w2v_words.append(remove_stopwords(Texts))
w2v_words[0]

'I bought Vitality canned dog food products good quality. The product looks like stew processed meat smells better. My Labrador finicky appreciates product better most.'

#### 用空白分割句子

In [19]:
w2v_words_list = []
for i in range(len(w2v_words)):
    w2v_words_list.append(w2v_words[i].split())
w2v_words_list[0]

['I',
 'bought',
 'Vitality',
 'canned',
 'dog',
 'food',
 'products',
 'good',
 'quality.',
 'The',
 'product',
 'looks',
 'like',
 'stew',
 'processed',
 'meat',
 'smells',
 'better.',
 'My',
 'Labrador',
 'finicky',
 'appreciates',
 'product',
 'better',
 'most.']

In [20]:
from gensim.models import Word2Vec
model = Word2Vec(w2v_words_list,vector_size=300,min_count=10,window=5) 
#vector size決定dimension數量(多少個feature)
#min count 表示詞出現多少次以上才計入
#window Maximum distance between the current and predicted word within a sentence.

In [21]:
model.wv['product'] #詞所代表的向量

array([ 1.72675233e-02,  2.88014889e-01, -6.02449030e-02, -1.15243951e-03,
       -1.29582703e-01, -2.50166565e-01,  1.45383939e-01,  4.64902043e-01,
        2.05339164e-01, -2.28777662e-01,  1.63434520e-01, -4.36167479e-01,
        1.48335367e-01, -1.16442516e-01, -2.19491974e-01,  7.38900900e-02,
        1.50169447e-01, -5.29956110e-02, -1.39451802e-01, -1.86075494e-01,
       -3.92811328e-01, -5.65155260e-02,  1.34550547e-02,  1.21119961e-01,
        1.66424036e-01,  2.30078056e-01, -2.38333046e-01, -5.31360321e-02,
       -1.36958897e-01, -4.73605573e-01, -1.70814840e-03, -1.24676391e-01,
        8.22380781e-02, -8.07563215e-03, -2.22525626e-01,  2.78399587e-01,
        2.64357209e-01, -1.58445701e-01,  1.25250476e-03,  5.38070463e-02,
       -3.11060071e-01,  1.19556747e-01, -4.96183395e-01, -4.54284668e-01,
        3.88917387e-01,  5.13749151e-03,  1.42344937e-01,  7.15091750e-02,
       -1.00792544e-02,  1.31137982e-01, -1.26689374e-01, -5.70178255e-02,
       -4.97880839e-02, -

In [22]:
print(f'總共取{len(model.wv)}個字,每個字有{len(model.wv[0])}個維度')

總共取5142個字,每個字有300個維度


In [23]:
len((model.wv.index_to_key)) #index為詞，column為feature(300)

5142

再來要計算每句話的平均Feature，先找出哪些詞有出現，再根據有出現的詞做平均

In [24]:
import numpy as np

In [25]:
feature_avg = np.zeros((len(w2v_words_list),len(model.wv[0]))) #總表
sentence_index = 0
w2v_index = model.wv.index_to_key                              # index為詞,col_name為300個 feature
for words in w2v_words_list:                                   # iter 每個 comment
    word_count = 0                                             # 計算重要的 word在所有 comment裡的次數
    feature_vec = np.zeros(len(model.wv[0]),dtype = 'float32') # 該 comment的feature 
    for word in words:                                         # iter每個 comment中的每個 word   
        if word in model.wv.index_to_key:                      # 若該 word有出現在 wv裡(表示相對重要)
            word_count += 1
            word_vector = model.wv[word]                       # 取出該 word在 w2v裡的300個 feature
            feature_vec = feature_vec + word_vector            # 記得這裡是(1,300)的運算
    avg = feature_vec / word_count                             # 計算該 comment裡，total feature/ 重要 word數量 = avg
    feature_avg[sentence_index] = avg                          # 將 avg儲存在總表裡 
    sentence_index += 1                                        # 移動到下一個 comment
df_w2v = pd.concat([sample_data,pd.DataFrame(feature_avg)],join='inner',axis=1)
df_w2v.head()

,Text,Score,0,1,2,3,4,5,6,7,...,290,291,292,293,294,295,296,297,298,299
0,I have bought several of the Vitality canned d...,1,-0.027959,0.245138,-0.043531,0.119486,-0.024511,-0.281336,0.070776,0.354023,...,-0.009458,0.216101,0.287688,-0.201719,0.197139,0.221891,-0.049641,-0.091092,0.139009,0.029154
1,Product arrived labeled as Jumbo Salted Peanut...,0,0.000311,0.226615,-0.049388,0.086323,-0.039253,-0.260148,0.064660,0.342760,...,-0.016318,0.140172,0.179877,-0.155793,0.109781,0.186728,-0.037544,-0.087230,0.092261,-0.036596
2,This is a confection that has been around a fe...,1,-0.012225,0.234607,-0.052463,0.112305,-0.035047,-0.265333,0.059659,0.357397,...,0.031916,0.149118,0.209803,-0.132396,0.098234,0.220340,0.035993,-0.069537,0.199743,-0.051426
3,If you are looking for the secret ingredient i...,0,-0.023246,0.241019,-0.006543,0.088321,-0.037592,-0.323466,0.075459,0.457880,...,-0.093818,0.167152,0.293918,-0.106076,0.206972,0.290518,0.124106,-0.040863,0.215021,-0.055944
4,Great taffy at a great price. There was a wid...,1,0.010424,0.189464,-0.062288,0.092318,-0.033870,-0.217970,0.058472,0.263572,...,0.023523,0.115695,0.144575,-0.100909,0.070387,0.150014,0.018097,-0.046799,0.144751,-0.026114


In [26]:
from sklearn import ensemble, preprocessing, metrics

In [27]:
def k_fold_crossvalidation(k:int ,data): #k = 切幾分
    
    #切割資料
    subset_size = int(len(data)/k)       #每個子集的長度
    avg_acc = 0                          #等等算平均用的
    total_acc = 0
    for i in range(k):
        start = subset_size * i
        end = start + subset_size
        valid = data[start:end]
        
        valid_x = valid.drop(['Score','Text'], axis = 1)
        valid_y = valid[['Score']]
                              
        if i == 0:
            train = data[end:]
        else:
            train = pd.concat([data[:start],data[end:]],axis=0,join ='inner',ignore_index=True)
        train_x = train.drop(['Score','Text'], axis = 1)
        train_y = train[['Score']]
    
        #開始種樹長森林
        forest = ensemble.RandomForestClassifier(n_estimators=100) #n_estimators決定有幾棵樹在森林
        forest_fit = forest.fit(train_x,train_y)
        valid_y_predicted = forest_fit.predict(valid_x)
        sub_acc = metrics.accuracy_score(valid_y,valid_y_predicted)
        print(f'valid為:{start},{end}')
        print(f'此subset_acc為{sub_acc}')

        total_acc += sub_acc
    avg_acc = total_acc / k
    return avg_acc

In [28]:
k_fold_crossvalidation(4,df_tfidf)

<ipython-input-27-ef38defc6100>:24: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  forest_fit = forest.fit(train_x,train_y)


valid為:0,2500
此subset_acc為0.8124


<ipython-input-27-ef38defc6100>:24: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  forest_fit = forest.fit(train_x,train_y)


valid為:2500,5000
此subset_acc為0.8048


<ipython-input-27-ef38defc6100>:24: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  forest_fit = forest.fit(train_x,train_y)


valid為:5000,7500
此subset_acc為0.7948


<ipython-input-27-ef38defc6100>:24: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  forest_fit = forest.fit(train_x,train_y)


valid為:7500,10000
此subset_acc為0.802


0.8035

In [29]:
k_fold_crossvalidation(4,df_w2v)

<ipython-input-27-ef38defc6100>:24: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  forest_fit = forest.fit(train_x,train_y)


valid為:0,2500
此subset_acc為0.7716


<ipython-input-27-ef38defc6100>:24: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  forest_fit = forest.fit(train_x,train_y)


valid為:2500,5000
此subset_acc為0.766


<ipython-input-27-ef38defc6100>:24: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  forest_fit = forest.fit(train_x,train_y)


valid為:5000,7500
此subset_acc為0.746


<ipython-input-27-ef38defc6100>:24: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  forest_fit = forest.fit(train_x,train_y)


valid為:7500,10000
此subset_acc為0.7592


0.7606999999999999